In [24]:
"""
Format dataset, we read the file for the desired subject, and parse the data to extract:
- samplingRate
- trialLength
- X, a M x N x K matrix, which stands for trial x chan x samples
                         the actual values are 160 x 15 x 2560
- y, a M vector containing the labels {0,1}

ref:
Dataset description: https://lampx.tugraz.at/~bci/database/002-2014/description.pdf
"""

import scipy.io as sio
import numpy as np


# prepare data containers
y = []
X = []


"""
trainingFileList = ['BBCIData/S14T.mat', 
                    'BBCIData/S13T.mat', 
                    'BBCIData/S12T.mat', 
                    'BBCIData/S11T.mat', 
                    'BBCIData/S10T.mat', 
                    'BBCIData/S09T.mat', 
                    'BBCIData/S08T.mat', 
                    'BBCIData/S07T.mat', 
                    'BBCIData/S06T.mat', 
                    'BBCIData/S05T.mat', 
                    'BBCIData/S04T.mat', 
                    'BBCIData/S03T.mat', 
                    'BBCIData/S02T.mat', 
                    'BBCIData/S01T.mat']

validationFileList = ['BBCIData/S14E.mat', 
                      'BBCIData/S13E.mat', 
                      'BBCIData/S12E.mat', 
                      'BBCIData/S11E.mat', 
                      'BBCIData/S10E.mat', 
                      'BBCIData/S09E.mat', 
                      'BBCIData/S08E.mat', 
                      'BBCIData/S07E.mat', 
                      'BBCIData/S06E.mat', 
                      'BBCIData/S05E.mat', 
                      'BBCIData/S04E.mat', 
                      'BBCIData/S03E.mat', 
                      'BBCIData/S02E.mat', 
                      'BBCIData/S01E.mat']
"""


trainingFileList = ['BBCIData/S04T.mat',
                    'BBCIData/S03T.mat', 
                    'BBCIData/S02T.mat', 
                    'BBCIData/S01T.mat']

validationFileList = ['BBCIData/S04E.mat',
                      'BBCIData/S03E.mat', 
                      'BBCIData/S02E.mat', 
                      'BBCIData/S01E.mat']



for i in range(len(trainingFileList)):
    # read file
    d1T = sio.loadmat(trainingFileList[i])
    d1E = sio.loadmat(validationFileList[i])
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = 5*samplingRate


    # run through all training runs
    for run in range(5):
        y.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X.append(rawData[:,start:end]) #15 x 2560


    # run through all validation runs (we do not discriminate at this point)
    for run in range(3):
        y.append(d1E['data'][0][run][0][0][2][0]) # labels
        timestamps = d1E['data'][0][run][0][0][1][0] # timestamps
        rawData = d1E['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X.append(rawData[:,start:end]) #15 x 2560

# arrange data into numpy arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X = np.array(X).astype(np.float32)
y = (np.array(y).flatten()-1).astype(np.int64)
print(X.shape)
print(y.shape)

# erase unused references
d1T = []
d1E = []


(640, 15, 2560)
(640,)


In [25]:
from braindecode.datautil.signal_target import SignalAndTarget
from braindecode.models.shallow_fbcsp import ShallowFBCSPNet
from torch import nn
from braindecode.torch_ext.util import set_random_seeds    
from torch import optim


idx = np.random.permutation(X.shape[0])

X = X[idx,:,:]
y = y[idx]

#print(X.shape)
#print(y.shape)

nb_train_trials = int(np.floor(5/8*X.shape[0]))


train_set = SignalAndTarget(X[:nb_train_trials], y=y[:nb_train_trials])
test_set = SignalAndTarget(X[nb_train_trials:], y=y[nb_train_trials:])

#train_set = SignalAndTarget(X[:nb_train_trials], y=y[:nb_train_trials])
#test_set = SignalAndTarget(X[nb_train_trials:nb_test_trials], y=y[nb_train_trials:nb_test_trials])

# Set if you want to use GPU
# You can also use torch.cuda.is_available() to determine if cuda is available on your machine.
cuda = False
set_random_seeds(seed=20170629, cuda=cuda)
n_classes = 2
in_chans = train_set.X.shape[1]
# final_conv_length = auto ensures we only get a single output in the time dimension
model = ShallowFBCSPNet(in_chans=in_chans, n_classes=n_classes,
                        input_time_length=train_set.X.shape[2],
                        final_conv_length='auto').create_network()
if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters())


In [26]:

from braindecode.torch_ext.util import np_to_var, var_to_np
from braindecode.datautil.iterators import get_balanced_batches
import torch.nn.functional as F
from numpy.random import RandomState
rng = RandomState((2017,6,30))
for i_epoch in range(30):
    i_trials_in_batch = get_balanced_batches(len(train_set.X), rng, shuffle=True,
                                            batch_size=15)
    # Set model to training mode
    model.train()
    for i_trials in i_trials_in_batch:
        # Have to add empty fourth dimension to X
        batch_X = train_set.X[i_trials][:,:,:,None]
        batch_y = train_set.y[i_trials]
        net_in = np_to_var(batch_X)
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(batch_y)
        if cuda:
            net_target = net_target.cuda()
        # Remove gradients of last backward pass from all parameters
        optimizer.zero_grad()
        # Compute outputs of the network
        outputs = model(net_in)
        # Compute the loss
        loss = F.nll_loss(outputs, net_target)
        # Do the backpropagation
        loss.backward()
        # Update parameters with the optimizer
        optimizer.step()

    # Print some statistics each epoch
    model.eval()
    print("Epoch {:d}".format(i_epoch))
    for setname, dataset in (('Train', train_set), ('Test', test_set)):
        # Here, we will use the entire dataset at once, which is still possible
        # for such smaller datasets. Otherwise we would have to use batches.
        net_in = np_to_var(dataset.X[:,:,:,None])
        if cuda:
            net_in = net_in.cuda()
        net_target = np_to_var(dataset.y)
        if cuda:
            net_target = net_target.cuda()
        outputs = model(net_in)
        loss = F.nll_loss(outputs, net_target)
        print("{:6s} Loss: {:.5f}".format(
            setname, float(var_to_np(loss))))
        predicted_labels = np.argmax(var_to_np(outputs), axis=1)
        accuracy = np.mean(dataset.y  == predicted_labels)
        print("{:6s} Accuracy: {:.1f}%".format(
            setname, accuracy * 100))

Epoch 0
Train  Loss: 0.67308
Train  Accuracy: 66.2%
Test   Loss: 1.09870
Test   Accuracy: 46.7%
Epoch 1
Train  Loss: 0.63049
Train  Accuracy: 67.0%
Test   Loss: 1.06700
Test   Accuracy: 47.9%
Epoch 2
Train  Loss: 0.57933
Train  Accuracy: 71.8%
Test   Loss: 1.06950
Test   Accuracy: 49.2%
Epoch 3
Train  Loss: 0.45585
Train  Accuracy: 76.8%
Test   Loss: 0.98647
Test   Accuracy: 50.4%
Epoch 4
Train  Loss: 0.39514
Train  Accuracy: 82.8%
Test   Loss: 0.98505
Test   Accuracy: 53.8%
Epoch 5
Train  Loss: 0.41111
Train  Accuracy: 81.2%
Test   Loss: 0.99206
Test   Accuracy: 52.9%
Epoch 6
Train  Loss: 0.47935
Train  Accuracy: 79.0%
Test   Loss: 1.15841
Test   Accuracy: 52.9%
Epoch 7
Train  Loss: 0.44434
Train  Accuracy: 83.5%
Test   Loss: 1.16125
Test   Accuracy: 52.5%
Epoch 8
Train  Loss: 0.23925
Train  Accuracy: 91.0%
Test   Loss: 0.94978
Test   Accuracy: 57.9%
Epoch 9
Train  Loss: 0.24268
Train  Accuracy: 91.5%
Test   Loss: 0.91686
Test   Accuracy: 57.9%
Epoch 10
Train  Loss: 0.18259
Train  Acc

In [ ]:
# Problem: RAM not big enough
# next session, manage batches through the hard drive
# add analytics on training performance